data collected April 25, 2020 on a DigitalOcean droplet running Ubuntu 18.04

In [21]:
import pandas
print('pandas',pandas.__version__)
import re
import time
from geoip import geolite2
import sys
print(sys.version)
import matplotlib.pyplot as plt

pandas 0.23.4
3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]


In [3]:
with open('gunicorn_error.log','r') as f:
    lines = f.readlines()

In [7]:
lines[0:10]

['[2020-04-25 21:25:05 +0000] [1] [DEBUG] Current configuration:\n',
 '  config: None\n',
 "  bind: ['0.0.0.0:5000']\n",
 '  backlog: 2048\n',
 '  workers: 2\n',
 '  worker_class: gthread\n',
 '  threads: 4\n',
 '  worker_connections: 1000\n',
 '  max_requests: 0\n',
 '  max_requests_jitter: 0\n']

In [12]:
valid_line = ''
revised_list_of_lines = []
for line in lines:
    if line.startswith('[2020-'):
        revised_list_of_lines.append(valid_line)
        valid_line = line
    else:
        valid_line += line
        
revised_list_of_lines = revised_list_of_lines[1:]

In [18]:
revised_list_of_lines[2:5]

['[2020-04-25 21:25:05 +0000] [1] [DEBUG] Arbiter booted\n',
 '[2020-04-25 21:25:05 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)\n',
 '[2020-04-25 21:25:05 +0000] [1] [INFO] Using worker: gthread\n']

In [40]:
list_of_dicts = []
for line in revised_list_of_lines:
    line_as_dict = {}
    line_as_list = line.split(' ')
    line_as_dict['date time'] = line_as_list[0] + ' ' + line_as_list[1]
    line_as_dict['pid'] = line_as_list[3].replace('[','').replace(']','')
    line_as_dict['msg type'] = line_as_list[4].replace('[','').replace(']','')
    line_as_dict['msg'] = ' '.join(line_as_list[5:]).strip()
    
    list_of_dicts.append(line_as_dict)

In [41]:
df = pandas.DataFrame(list_of_dicts)
df.shape

(2673, 4)

In [42]:
# https://strftime.org/
df['datetime'] = pandas.to_datetime(df['date time'], format='[%Y-%m-%d %H:%M:%S')
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
df.drop('date time', inplace=True, axis=1)

In [45]:
df.head(10)

,msg,msg type,pid,datetime
0,Current configuration:\n config: None\n bind...,DEBUG,1,2020-04-25 21:25:05
1,Starting gunicorn 20.0.4,INFO,1,2020-04-25 21:25:05
2,Arbiter booted,DEBUG,1,2020-04-25 21:25:05
3,Listening at: http://0.0.0.0:5000 (1),INFO,1,2020-04-25 21:25:05
4,Using worker: gthread,INFO,1,2020-04-25 21:25:05
5,Booting worker with pid: 8,INFO,8,2020-04-25 21:25:05
6,Booting worker with pid: 9,INFO,9,2020-04-25 21:25:05
7,"2 workers\nUnable to drop ""data""; no such tabl...",DEBUG,1,2020-04-25 21:25:05
8,[trace],INFO,8,2020-04-25 21:25:09
9,[trace],INFO,8,2020-04-25 21:25:09


In [49]:
sum(df['msg']=='[trace]')

76

In [47]:
df['pid'].value_counts()

9    1431
8    1228
1      14
Name: pid, dtype: int64

In [46]:
df['msg type'].value_counts()

DEBUG    1675
INFO      952
ERROR      46
Name: msg type, dtype: int64

In [44]:
df['msg']

0       Current configuration:\n  config: None\n  bind...
1                                Starting gunicorn 20.0.4
2                                          Arbiter booted
3                   Listening at: http://0.0.0.0:5000 (1)
4                                   Using worker: gthread
5                              Booting worker with pid: 8
6                              Booting worker with pid: 9
7       2 workers\nUnable to drop "data"; no such tabl...
8                                                 [trace]
9                                                 [trace]
10                                                [trace]
11                                                [trace]
12                                                [trace]
13                                                [trace]
14                                                [trace]
15                                                [trace]
16                                                [trace]
17            